# Workflow Examples 


`xclim` is built on very powerful multiprocessing and distributed computation libraries, notably `xarray` and `dask`.

`xarray` is a python package making it easy to work with n-dimensional arrays. It labels axes with their names `[time, lat, lon, level]` instead of indices `[0,1,2,3]`, reducing the likelihood of bugs and making the code easier to understand. One of the key strengths of `xarray` is that it knows how to deal with non-standard calendars (we're looking at you, "`360_days`") and can easily resample daily time series to weekly, monthly, seasonal or annual periods.  Finally, `xarray` is tightly inegrated with `dask`, a package that can automatically parallelize operations.

The following are a few examples to consult when using `xclim` to subset netCDF arrays and compute climate indicators, taking advantage of the parallel processing capabilities offered by `xarray` and `dask`. For more information about these projects, please see their documentation pages:

* [xarray documentation](https://xarray.pydata.org/en/stable/)
* [dask documentation](https://docs.dask.org/en/stable/)

## Environment configuration

In [ ]:
# Imports for xclim and xarray
import xclim as xc
import numpy as np
import xarray as xr
xr.set_options(display_style='html')

# File handling libraries
import time
import tempfile
from pathlib import Path

# Output folder
output_folder = Path(tempfile.mkdtemp()) 

## Setting up the Dask client: parallel processing

This step is not mandatory as dask will fall back to its "single machine scheduler" if a `Client` is not created (the case is this notebook). However, this default scheduler doesn't allow you to set the number of threads or a memory limit and doesn't start the dashboard, which can be quite useful to understand your task's progress.

The `xarray` library will automatically connect to the dask scheduler when needed.

The dashboard link lets you see in real time how busy those workers are. More info : [dask distributed documentation](https://distributed.dask.org/en/latest/).

```python
from distributed import Client

# Depending on your workstation specifications, you may need to adjust these values.
# On a single machine, n_workers=1 is usually better.
client=Client(n_workers=1, threads_per_worker=4, memory_limit="4GB") 
client
```

<table style="border: 2px solid white;">
<tr>
<td style="vertical-align: top; border: 0px solid white">
<h3 style="text-align: left;">Client</h3>
<ul style="text-align: left; list-style: none; margin: 0; padding: 0;">
  <li><b>Scheduler: </b>tcp://127.0.0.1:36783</li>
  <li><b>Dashboard: </b><a href='http://127.0.0.1:35847/status' target='_blank'>http://127.0.0.1:35847/status</a>
</ul>
</td>
<td style="vertical-align: top; border: 0px solid white">
<h3 style="text-align: left;">Cluster</h3>
<ul style="text-align: left; list-style:none; margin: 0; padding: 0;">
  <li><b>Workers: </b>1</li>
  <li><b>Cores: </b>4</li>
  <li><b>Memory: </b>4.00 GB</li>
</ul>
</td>
</tr>
</table>

## Creating xarray datasets

To open a netCDF file with `xarray`, we use `xr.open_dataset(<path to file>)`. By default, the entire file is stored in one chunk, so there is no parallelism. To trigger parallel computations, we need to explicitly specify the **chunk size**. 

<div class="alert alert-info">

In this example, instead of opening a local file, we pass an *OPeNDAP* url to xarray. It retrieves the data automatically. Notice also that opening the dataset is quite fast. In fact, the data itself has not been downloaded yet, only the coordinates and the metadata. The downloads will be triggered only when the values need to be accessed directly.

</div>

`dask`'s parallelism is based on memory chunks. We need to tell `xarray` to split our netCDF array into chunks of a given size, and operations on each chunk of the array will automatically be dispatched to the workers. 

In [ ]:
data_url = 'https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/birdhouse/ouranos/cb-oura-1.0/ACCESS1-3/rcp85/day/ACCESS1-3_rcp85_day_allvars.ncml'

In [ ]:
# Chunking in memory along the time dimension.
# Note that the data type is a 'dask.array'. xarray will automatically use the dask scheduler.
ds = xr.open_dataset(data_url, chunks={'time': 365, 'lat': 84, 'lon': 75}, drop_variables=['ts', 'time_vectors'])
ds

In [ ]:
print(ds.tasmin.chunks)

## Multi-file datasets

NetCDF files are often split into periods to keep file size manageable. A single dataset can be split in dozens of individual files. `xarray` has a function `open_mfdataset` that can open and aggregate a list of files and construct a unique *logical* dataset. `open_mfdataset` can aggregate files over coordinates (time, lat, lon) and variables. 

* Note that opening a multi-file dataset automatically chunks the array (one chunk per file).
* Note also that because `xarray` reads every file metadata to place it in a logical order, it can take a while to load. 

In [ ]:
## Create multi-file data & chunks 
# ds = xr.open_mfdataset('/path/to/files*.nc')

## Subsetting and selecting data with xarray
xclim defines useful subsetting utilities in its [`subset`](../api.rst#module-xclim.subset) submodule. Those are presented in more details in the next example ([Subsetting with xclim](subset.ipynb)). Here, we will reduce the size of our data using the methods implemented in xarray ([docs here](http://xarray.pydata.org/en/stable/indexing.html)).

In [ ]:
ds2 = ds.sel(lat=slice(50, 45), lon=slice(-70, -65), time=slice('2090', '2100'))
ds2.tasmin

In [ ]:
ds3 = ds.sel(lat=46.8, lon=-71.22, method='nearest').sel(time='1993')
ds3.tasmin

## Climate index calculation & resampling frequencies

`xclim` has two layers for the calculation of indicators. The bottom layer is composed of a list of functions that take one or more `xarray.DataArray`'s as input and return an `xarray.DataArray` as output. You'll find these functions in `xclim.indices`. The indicator's logic is contained in this function, as well as potential unit conversions, but it doesn't perform any data consistency checks (like if the time frequency is daily), and doesn't not adjust the meta data of the output array. 

The second layer are class instances that you'll find organized by *realm*. So far, there are three realms available in `xclim.atmos`, `xclim.seaIce` and `xclim.land`, the first one being the most exhaustive. Before running computations, these classes check if the input data is a daily average of the expected variable:

1. If an indicator expects a daily mean and you pass it a daily max, a `warning` will be raised. 
2. After the computation, it also checks the number of values per period to make sure there are not missing values or `NaN` in the input data. If there are, the output is going to be set to `NaN`.  Ex. : If the indicator performs a yearly resampling but there are only 350 non-`NaN` values in one given year in the input data, that year's output will be `NaN`.
3. The output units are set correctly as well as other properties of the output array, complying as much as possible with CF conventions. 

For new users, we suggest you use the classes found in `xclim.atmos` and others. If you know what you're doing and you want to circumvent the built-in checks, then you can use the `xclim.indices` directly. 

Almost all `xclim` indicators convert daily data to lower time frequencies, such as seasonal or annual values. This is done using `xarray.DataArray.resample` method. Resampling creates a grouped object over which you apply a reduction operation (e.g. mean, min, max). The list of available frequency is given in the link below, but the most often used are: 

- `YS`: annual starting in January
- `YS-JUL`: annual starting in July
- `MS`: monthly
- `QS-DEC`: seasonal starting in December

More info about this specification can be found in [pandas' documentation](http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-offset-aliases)

Note - not all offsets in the link are supported by cftime objects in `xarray`.


In the example below, we're computing the **annual maximum temperature of the daily maximum temperature (tx_max)**.

In [ ]:
out = xc.atmos.tx_max(ds2.tasmax, freq='YS')
out

<div class="alert alert-info">

If you execute the cell above, you'll see that this operation is quite fast. This a feature coming from dask. Read *Lazy computation*  further down.

</div>

### Comparison of **atmos** vs **indices** modules
Using the `xclim.indices` module performs not checks and only fills the `units` attribute.

In [ ]:
out = xc.indices.tx_days_above(ds2.tasmax, thresh='30 C', freq='YS')
out

With `xclim.atmos`, checks are performed and many CF-compliant attributes are added:

In [ ]:
out = xc.atmos.tx_days_above(ds2.tasmax, thresh='30 C', freq='YS')
out

In [ ]:
# We have created an xarray data-array - We can insert this into an output xr.Dataset object with a copy of the original dataset global attrs
dsOut = xr.Dataset(attrs=ds2.attrs)

# Add our climate index as a data variable to the dataset
dsOut[out.name] = out
dsOut

## *Lazy* computation - Nothing has been computed so far !

If you look at the output of those operations, they're identified as `dask.array` objects. What happens is that `dask` creates a chain of operations that when executed, will yield the values we want. We have thus far only created a schedule of tasks with a small preview and not done any actual computations. You can trigger computations by using the `load` or `compute` method, or writing the output to disk via `to_netcdf`. Of course, calling `.plot()` will also trigger the computation.

In [ ]:
%%time
output_file = output_folder / 'test_tx_days_above.nc'
dsOut.to_netcdf(output_file)

*(Times may of course vary depending on the machine and the Client settings)*

### Performance tips
#### Optimizing the chunk size

You can improve performance by being smart about chunk sizes. If chunks are too small, there is a lot of time lost in overhead. If chunks are too large, you may end up exceeding the individual worker memory limit.

In [ ]:
print(ds2.chunks['time'])

In [ ]:
# rechunk data in memory for the entire grid  
ds2c = ds2.chunk(chunks={'time': 365})
print(ds2c.chunks['time'])

In [ ]:
%%time
out = xc.atmos.tx_days_above(ds2c.tasmax, thresh='30 C', freq='YS')
dsOut = xr.Dataset(data_vars=None, coords=out.coords, attrs=ds.attrs)
dsOut[out.name] = out

output_file = output_folder / 'test_tx_max.nc'
dsOut.to_netcdf(output_file)

#### Loading the data in memory
If the dataset is relatively small, it might be more efficient to simply load the data into the memory and use  numpy arrays instead of dask arrays.

In [ ]:
# ds4 = ds3.load()